In [ ]:
import torchio as tio
import monai
from monai.transforms import (
    AddChanneld,
    LoadImage,
    LoadImaged,
    Orientationd,
    Rand3DElasticd,
    RandAffined,
    Spacingd,
    ToTensord,
    RandAffine
)
import torchvision as tv
import os
import numpy as np
import custom_models
import torch as t
from copy import deepcopy
import loss_module
import time
import matplotlib.pyplot as plt
from SVR_Preprocessor import Preprocesser
import SimpleITK as sitk

In [ ]:
pixdim = (2.0,2.0,2.0)

src_folder = "sample_data"
prep_folder = "cropped_images"
src_folder = "sample_data"
result_folder = os.path.join("results","two_stacks_first_outlier_removal")

stack_filenames = ["10_3T_nody_001.nii.gz",
                
                "14_3T_nody_001.nii.gz",
                
                "21_3T_nody_001.nii.gz",
                
                "23_3T_nody_001.nii.gz"]
device = "cpu"
mask_filename = "mask_10_3T_brain_smooth.nii.gz"
mode =  "bilinear"
tio_mode = "welch"

svr_preprocessor = Preprocesser(src_folder, prep_folder, result_folder, stack_filenames, mask_filename, device, mode, tio_mode=tio_mode)
        
#fixed_images, stacks = svr_preprocessor.preprocess_stacks_and_common_vol(pixdim)

#tio_image = svr_preprocessor.monai_to_torchio(stacks[0])

In [ ]:
stacks = svr_preprocessor.load_stacks()
tio_image = svr_preprocessor.monai_to_torchio(stacks[0])

In [ ]:
sitk_img = tio_image.as_sitk()

In [ ]:
rotation_center = (100, 100, 100)
axis = (0,0,1)
angle = np.pi/2.0
translation = (1,2,3)
scale_factor = 2.0
similarity = sitk.Similarity3DTransform(scale_factor, axis, angle, translation, rotation_center)

affine = sitk.AffineTransform(3)
affine.SetMatrix(similarity.GetMatrix())
affine.SetTranslation(similarity.GetTranslation())
affine.SetCenter(similarity.GetCenter())

# Apply the transformations to the same set of random points and compare the results.

In [ ]:
def sitk_affine_transform(tio_image:tio.Image, affine_matr:t.tensor)->t.tensor:
    sitk_image = tio_image.as_sitk()

    rotation = affine_matr[:3,:3].ravel().tolist()
    translation = affine_matr[:3,3].tolist()
    affine = sitk.AffineTransform(rotation,translation)

    reference_image = sitk_image
    interpolator = sitk.sitkWelchWindowedSinc
    default_value = 0

    resampled =  sitk.Resample(sitk_image,reference_image,affine,interpolator,default_value)

    tensor = t.permute(t.tensor(sitk.GetArrayFromImage(resampled)),(2,1,0))

    tensor = tensor.unsqueeze(0)
    return tensor

In [ ]:
affine_matr = t.eye(4)
sitk_transformed = sitk_affine_transform(tio_image,affine_matr)

In [ ]:
sitk_transformed.shape